Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [12]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [13]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [14]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print('training data size: %s %s' % (train_size, train_text[:64]))
print('validation data size: %s %s' % (valid_size, valid_text[:64]))

training data size: 99999000 ons anarchists advocate social relations based upon voluntary as
validation data size: 1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [15]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        #print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))
print('vocabulary size: %s' % vocabulary_size)

1 26 0 0
a z  
vocabulary size: 27


Function to generate a training batch for the LSTM model.

In [27]:
batch_size=64
num_unrollings=10

day_debug = True

# DAY
#
# This is important to understand.  Our NN needs a constant sized vector with each input.  We are
# providing that here.  As the video says, just as convolution lets us use the same weight parameters
# at different parts of the image, a recurrent neural net lets us use the same weights at different
# points in time (or rather, different points in the input sequence).
#
# The notion of "unrollings" is that a recurrent NN has it's output connected to it's input, but really
# the way to think about it is over time where the output of time t-1 is input to time t.  That way
# of looking at it is like "unrolling" the recurrent NN over time so it is understood more as a
# sequence of copies of the NN.  
# In this case, we are going to be feeding in sequences that are 10 long, so we will in effect
# create 10 LSTM cells (which are really just a NN) and hook the output of LSTM cell t with inputs
# from input_sub_t and also the output of LSTM cell t-1.

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        if day_debug:
            print('BatchGenerator init: \n  textsize: %s\n  batch_size: %s\n  num_unrollings: %s' % 
                  (len(text), batch_size, num_unrollings))
            print("BatchGenerator init -- text starts with: %s" % text[:128])
            print("BatchGenerator init -- segment=%s" % segment)
            print("BatchGenerator init -- _cursor=%s" % self._cursor)
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data.
        It will be in the form of a 1 hot encoding here (but not in problem 2)"""
        # DAY
        # this is quite confusing, but batch ends up being  of dimension
        # (batch_size, vocabulary_size) -- so it is 1 char (one-hot encoded) from
        # every _cursor location (there are batch_size cursor locations.  
        # Since the cursor locations are not consecutive, these chars in batch
        # are not consecutive.  However, in the next step at next(), the 
        # char will be appended with the next consecutive char.
        # Uncomment the print line below to see this.
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        #print(batches2string([batch]))
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    # DAY
    # This mangles the real batch structure in the interest of readability, but
    # by doing so, makes your understanding of batches wrong.
    # See my 'honest_batches2string' below which gives you a better
    # understanding of the batch format.
    
    # batches has dimensions (num_unrollings, batch_size, vocabulary_size)
    s = [''] * batches[0].shape[0]  # batches[0].shape[0] will end up being same as batch_size
    for b in batches: # there will be num_unrollings of these...
        # each b (batch) is 64 by 27
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

def honest_batches2string(batches):
    import pprint
    output = []
    for b_index, b in enumerate(batches):  # there will be 'num_unrollings' of these
        output.append(list())
        for one_hot_index, one_hot in enumerate(b):  # there will be 'batch_size' of these
            output[b_index].append(characters([one_hot]))
    return pprint.pformat(output)
            
        

if day_debug:
    # DAY
    # Notice how this output is structured. Subsequent batches are related to each other in that
    # you can continue reading the text from batch1[0] to batch2[0]  (similarly from
    # batch1[20] to batch2[20])
    my_text = "abcdefghijklmnopqrstuvwxyz"
    my_batches = BatchGenerator(my_text.lower(), 4, 2)
    if False:
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
    else:
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))

    my_text = "Four score and seven years ago our fathers brought forth on this continent, a new \
    nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. Now \
    we are engaged in a great civil war, testing whether that nation, or any nation so conceived and \
    so dedicated, can long endure. We are met on a great battle-field of that war. We have come to \
    dedicate a portion of that field, as a final resting place for those who here gave their lives that \
    that nation might live. It is altogether fitting and proper that we should do this.\
    But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- \
    this ground. The brave men, living and dead, who struggled here, have consecrated it, far above \
    our poor power to add or detract. The world will little note, nor long remember what we say here, \
    but it can never forget what they did here. It is for us the living, rather, to be dedicated here \
    to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for \
    us to be here dedicated to the great task remaining before us -- that from these honored dead we \
    take increased devotion to that cause for which they gave the last full measure of devotion -- that \
    we here highly resolve that these dead shall not have died in vain -- that this nation, under God, \
    shall have a new birth of freedom -- and that government of the people, by the people, for the people, \
    shall not perish from the earth."
    my_batches = BatchGenerator(my_text.lower(), 10, 5)
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    # Also, here is how a batch *really* looks (an batch_size array of one-hot encodings)
    temp = my_batches.next()
    print('Dimensions of the batch (num_unrollings, batch_size, vocab_size): (%d, %d, %d)' % 
          (len(temp), len(temp[0]), len(temp[0][0])))
    print('='*80)



train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print('WARNING: the following printed lines are not indicative of the structure that goes into the model')
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))


BatchGenerator init: 
  textsize: 26
  batch_size: 4
  num_unrollings: 2
BatchGenerator init -- text starts with: abcdefghijklmnopqrstuvwxyz
BatchGenerator init -- segment=6
BatchGenerator init -- _cursor=[1, 7, 13, 19]
[[['a'], ['g'], ['m'], ['s']],
 [['b'], ['h'], ['n'], ['t']],
 [['c'], ['i'], ['o'], ['u']]]
[[['c'], ['i'], ['o'], ['u']],
 [['d'], ['j'], ['p'], ['v']],
 [['e'], ['k'], ['q'], ['w']]]
[[['e'], ['k'], ['q'], ['w']],
 [['f'], ['l'], ['r'], ['x']],
 [['g'], ['m'], ['s'], ['y']]]
[[['g'], ['m'], ['s'], ['y']],
 [['h'], ['n'], ['t'], ['z']],
 [['i'], ['o'], ['u'], ['a']]]
[[['i'], ['o'], ['u'], ['a']],
 [['j'], ['p'], ['v'], ['b']],
 [['k'], ['q'], ['w'], ['c']]]
[[['k'], ['q'], ['w'], ['c']],
 [['l'], ['r'], ['x'], ['d']],
 [['m'], ['s'], ['y'], ['e']]]
BatchGenerator init: 
  textsize: 1532
  batch_size: 10
  num_unrollings: 5
BatchGenerator init -- text starts with: four score and seven years ago our fathers brought forth on this continent, a new     nation, conceived i

In [17]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0  # prediction is in column format, so it must be indexed by [0]
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

_*DAY: Note that I have modified this to comment/understand*_

In [18]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1)) # [27, 64]
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))       # [64, 64]
    ib = tf.Variable(tf.zeros([1, num_nodes]))                                     # [1, 64]
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #notice the 'trainable=False'
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #notice the 'trainable=False'
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    # The LSTM
    #
    # In the code above, batch_size (bs) and num_nodes (nn) are both 64 (they don't have to be equal)
    # 27 is the vocabulary size.
    #
    # input_gate(bs, nn)   = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # forget_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # output_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # update(bs, nn)       = tanh(    input(bs, 27) * cx(27, nn) + output(bs, nn) * cm(nn, nn) + cb(1, nn) )
    #
    # output(bs, nn) = output_gate(bs, nn) * tanh( state(bs, nn) )
    # state(bs, nn)  = forget_gate(bs, nn) * state(bs, nn)  +  input_gate(bs, nn) * update(bs, nn)

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            # These 11 elements of train_data will be pulled in from feed_dict
            # Note that usually we have seen feed_dict specified as {var_name: value}
            # but in this case, since these 11 array elements don't have a var_name, the 
            # feed_dict will use the tensorflow object as the key instead, i.e.
            # feed_dict={<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>: value}
            # See below where the feed_dict is prepared before calling session.run.
            tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size])) #pulled in from feed_dict
    # train_data now has the shape (11, 64, 27), or (num_unrollings, batch_size, vocabulary_size)
    # and sequential text from the original text input is 'striped' across the first dimension (11)
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step. (Brilliant!)

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs: # since train_inputs is num_unrollings=10 long, this will create 10 LSTM cells
        output, state = lstm_cell(i, output, state)
        outputs.append(output) # at each iter of the lstm_cell, append the character it predicted to outputs.

    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    #   Note that all 10 unrollings are done before the optimizer comes in and looks at the
    #   output sequence of 10 chars vs. the label sequence of 10 chars and then calculates
    #   the gradients and adjusts the parameters.  Then in the next step another 10 characters
    #   will be predicted.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    # DAY: this clipping below is the hack to prevent exploding gradients 
    #(LSTM was the elegant solution to prevent vanishing gradient)
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    # (nothing here is triggered in the training)
    #     first, variables
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    #     reset zeros out saved_sample_output and saved_sample_state
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    #     Define one lstm_cell with no unrolling (will be used for sampling from the trained model)
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    #     Define the next prediction (but make sure dependencies are calculated first)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Note that I've modified this with comments to understand

In [10]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next() #
        # batches is of dimension (num_unrollings, batch_size, vocabulary_size) (11, 64, 27)
        # where sequential text from the input is "striped" across the unrollings.  For 
        # example, if 'char1' stands for the first character in the original text (one-hot encoded), 
        # batches looks like this (assuming 'segment' is 15000):
        # [                                        # there are 'num_unrollings' rows
        #   [char1,  char15000,  char 30000, ...], # each row is 'batch_size'
        #   [char2,  char15001,  char 30001, ...],
        #   ...
        #   [char11, char15010,  char 30010, ...]
        # ]
        # when we call train_batches.next(), the next 'batches' will look like this:
        # [                                        # there are 'num_unrollings' rows
        #   [char11, char15010,  char 30010, ...], # each row is 'batch_size'
        #   [char12, char15011,  char 30011, ...],
        #   ...
        #   [char21, char15020,  char 30020, ...]
        # ]
        # it might look like a bug that the second 'batches' repeats char11, char1510, etc.
        # but it is not a bug.  in the first 'batches', char11 was included only as the label
        # needed for the 10th entry (char10). (LSTM takes char10 in as an input and expects char11
        # as the true label of the output).  So -- char11 was never put into the LSTM_cell in the
        # first 'batches' -- it is only used as a label, so it need to be included as the first
        # item in the second 'batches' so that in can now be an input into an LSTM cell.
        #
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            # Normally we see feed_dict={var_name: value}, but here we don't have the var_names
            # for the training data batches in the graph definition (it is an array of tensors)
            # so instead, we use the tensorflow object itself (from the graph definition, in
            # train_data[i]) as the key in the feed_dict entries.
            
        _, l, predictions, lr = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())  # start with a random character
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.300142 learning rate: 10.000000
Minibatch perplexity: 27.12
lqy baquzzqshluwe  uneejomcunzn vxh  ts o op    f niyemiynetuelhtdelj cqvtoe hd 
nffghw e sklcio yvcdroaydod wr  tlcgq geesnqojqwcu kececysr ehpqliklpsudzatfma z
shfa gjc  clvvwava w lx ikyzkiq h yuuerbtvsrshw pnbifupptpep  m uqdarmesi bn wm 
xknulgrvtart  a rqaifcovbov x kje emeet e fabfewu s mhys i dv egetvdp hhesxsiouz
arunxakmpatiwrxzecdfsqrjk goyufbtcsqiuauyrpeztoiejc dcmaeandizu pem ffi  snnft c
Validation set perplexity: 20.34
Average loss at step 100: 2.622735 learning rate: 10.000000
Minibatch perplexity: 11.70
Validation set perplexity: 10.89
Average loss at step 200: 2.282770 learning rate: 10.000000
Minibatch perplexity: 8.75
Validation set perplexity: 8.75
Average loss at step 300: 2.124059 learning rate: 10.000000
Minibatch perplexity: 7.55
Validation set perplexity: 8.09
Average loss at step 400: 2.012910 learning rate: 10.000000
Minibatch perplexity: 7.60
Validation set per

KeyboardInterrupt: 

# DAY - small insertion to understand the character sampling

In [33]:
# DAY - small insertion to understand how the character sampling works

# first, let's run the optimization of the model for just a short while:
num_steps = 500
update_frequency = num_steps/20

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            # train_data is an array of tf.placeholders of len 'num_unrollings' +1
            # so feed_dict now has entries with a tf.placeholder as key and a batch as value


        _, l, predictions, lr = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)

        if step % update_frequency == 0:
            print('.', end='')
            print('step %d:\n    feed_dict=%s\n    train_prediction=%s' % (step, feed_dict, train_prediction))

        mean_loss += l
# OK, now that the model is just a little trained, report out some stats
    mean_loss = mean_loss / num_steps
    print('\nAverage loss over the 500 steps was %f' % mean_loss)
    labels = np.concatenate(list(batches)[1:])
    print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))

    print('Perplexity of last minibatch (i.e.how confused was the model in choosing \
the next char -- ie roughly \
how many chars with equal probability did it have to chose from) was %.2f', float(
                np.exp(logprob(predictions, labels))))
    
# now do the sampling:
    print('\nSampling Explanation:')
    print('=' * 80)
    rd = random_distribution()
    print('1. random_distribution yields vocabulary_size(27) random #s %s' % rd)
    feed = sample(rd)
    print('2. take a sample from that in a one-hot format, call it "feed": %s' % feed)
    print('3. characters converts to a char.  character(feed)=%s' % characters(feed))
    sentence = characters(feed)[0]
    print('4. the beginning of your "sentence" (really a line) is this char: %s' % sentence)
    reset_sample_state.run()
    print('5. evaluate the "reset_sample_state" formula defined above.  It zeros out \
the saved_sample_output and saved_sample_state.  This is important because we want \
predictions from the beginning of the line (i.e. no previous state).\n\
The thing that is "remembered" in the model and doing the predictions are the parameters, \
(i.e. ix, im, ib, ox, om, ob, fx, fm, fb, cx, cm, cb, w, b), not the state or output -- the \
state and output are specific to the minibatch inputs that have preceeded.')
    print('6. FOR EACH of the next 79 characters in the line, make a prediction as follows:')
    prediction = sample_prediction.eval({sample_input: feed})
    print('   6a. Evaluate a "prediction" variable by "eval"ing the "sample_prediction" variable above. \
This triggers the evaluation of a single LSTM cell (not an unrolling of 10) which uses the trained \
parameters to output a prediction for the next character.  This output is turned into a logit with the \
final matmul with w and b and then a softmax is taken which is the prediction.\n\
Here is the prediction after the first character: %s' % prediction)
    feed = sample(prediction)
    print('   6b. the sample() function takes that prediction, which is a vector of probabilities, and \
returns a sampled character such that high probability characters are more likely (but not guaranteed) \
to be chosen.  Here is what it chooses for the prediction above: %s' % feed)
    sentence += characters(feed)[0]
    print('Now convert that one-hot encoding to a char and append to the end of the sentence: %s' % sentence)
    
    print('\n Now here is all of that together:')
          
    print('=' * 80)
    for _ in range(5):  # sample 5 lines, each line will be (below) 79 chars
        for _ in range(79):
            # trigger evaluation of one lstm_cell and capture the softmax(logit) output as prediction
            prediction = sample_prediction.eval({sample_input: feed})
            # convert prediction into a character
            feed = sample(prediction)
            # append the character onto our string
            sentence += characters(feed)[0]
        print(sentence)
        # DAY, I moved these 3 lines to the bottom of the loop so I continue the string from above the loop
        feed = sample(random_distribution()) # start with a random character
        sentence = characters(feed)[0]
        reset_sample_state.run()
    print('=' * 80)

Initialized
.step 0:
    feed_dict={<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>: array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), <tf.Tensor 'Placeholder_5:0' shape=(64, 27) dtype=float32>: array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]]), <tf.Tensor 'Placeholder_10:0' shape=(64, 27) dtype=float32>: array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Answer 1
========

1. I added tensorboard hooks (they aren't quite useful yet
2. Concatenate the 4 gates together using tf.concat on the 1th dimension (not the 0th)
3. Do the matmul on the concatenated gates
4. Split the result back out
5. Apply either sigmoid or tanh activation function as appropriate.

---

In [19]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    with tf.name_scope('LSTM_cell') as lstm_cell_scope:
        with tf.name_scope('input_gate') as input_gate_scope:
            # Input gate: input, previous output, and bias.
            ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1)) # [27, 64]
            im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))       # [64, 64]
            ib = tf.Variable(tf.zeros([1, num_nodes]))                                     # [1, 64]
        with tf.name_scope('forget_gate') as forget_gate_scope:
            # Forget gate: input, previous output, and bias.
            fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
            fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            fb = tf.Variable(tf.zeros([1, num_nodes]))
        with tf.name_scope('memory_cell') as memory_cell_scope:
            # Memory cell: input, state and bias.                             
            cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
            cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            cb = tf.Variable(tf.zeros([1, num_nodes]))
        with tf.name_scope('output_gate') as output_gate_scope:
            # Output gate: input, previous output, and bias.
            ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
            om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
            ob = tf.Variable(tf.zeros([1, num_nodes]))
            
            
        # Reduce the input/output matmuls from 4 to 1 each by concatenating the 4 gates
        concatx = tf.concat(1, [ix, fx, cx, ox])
        concatm = tf.concat(1, [im, fm, cm, om])
        concatb = tf.concat(1, [ib, fb, cb, ob])
        
        
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        with tf.name_scope(lstm_cell_scope):
            # Instead of these 4 matmuls, do the one concatenated matmul and then split results
            #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
            #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
            #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
            #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)

            concatmatmul = tf.matmul(i, concatx) + tf.matmul(o, concatm) + concatb
            input_gate, forget_gate, update, output_gate = tf.split(1, 4, concatmatmul)
            input_gate = tf.sigmoid(input_gate)
            forget_gate = tf.sigmoid(forget_gate)
            output_gate = tf.sigmoid(output_gate)
            state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state
    
    # The LSTM
    #
    # In the code above, batch_size (bs) and num_nodes (nn) are both 64 (they don't have to be equal)
    # 27 is the vocabulary size.
    #
    # input_gate(bs, nn)   = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # forget_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # output_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # update(bs, nn)       = tanh(    input(bs, 27) * cx(27, nn) + output(bs, nn) * cm(nn, nn) + cb(1, nn) )
    #
    # output(bs, nn) = output_gate(bs, nn) * tanh( state(bs, nn) )
    # state(bs, nn)  = forget_gate(bs, nn) * state(bs, nn)  +  input_gate(bs, nn) * update(bs, nn)

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            # These 11 elements of train_data will be pulled in from feed_dict
            # Note that usually we have seen feed_dict specified as {var_name: value}
            # but in this case, since these 11 array elements don't have a var_name, the 
            # feed_dict will use the tensorflow object as the key instead, i.e.
            # feed_dict={<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>: value}
            # See below where the feed_dict is prepared before calling session.run.
            tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size])) #this will be pulled in from feed_dict
    # train_data now has the shape (11, 64, 27), or (num_unrollings, batch_size, vocabulary_size)
    # and sequential text from the original text input is 'striped' across the first dimension (11)
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step. (Brilliant!)


    # Unrolled LSTM loop.
    with tf.name_scope(lstm_cell_scope):
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs: # since train_inputs is num_unrollings=10 long, this will create 10 LSTM cells
            output, state = lstm_cell(i, output, state)
            outputs.append(output)  # at each iter of the lstm_cell, append the character it predicted to outputs.

    # State saving across unrollings.
    #
    # DAY:  I am surprised that the output and state are saved across unrollings given that the
    #       way the batches are done, the unrollings are not consecutive, so the state at the beginning
    #       of a step should be empty rather than having the state from the last 10 characters of
    #       the last step.
    # ??
    #
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))
        tf.scalar_summary('loss', loss)


    # Optimizer.
    #   Note that all 10 unrollings are done before the optimizer comes in and looks at the
    #   output sequence of 10 chars vs. the label sequence of 10 chars and then calculates
    #   the gradients and adjusts the parameters.  Then in the next step another 10 characters
    #   will be predicted.
    with tf.name_scope("Optimizer"):
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(
            10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        # DAY: this clipping below is the hack to prevent exploding gradients 
        #(LSTM was the elegant solution to prevent vanishing gradient)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(
            zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    # (nothing here is triggered in the training)
    #     first, variables
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    #     reset zeros out saved_sample_output and saved_sample_state
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    #     Define one lstm_cell with no unrolling (will be used for sampling from the trained model)
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    #     Define the next prediction (but make sure dependencies are calculated first)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


In [20]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    merged = tf.merge_all_summaries()
    #writer = tf.train.SummaryWriter('logs', graph=session.graph) #use relative dirname
    writer = tf.train.SummaryWriter('logs') #use relative dirname
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr, merged_output = session.run(
              [optimizer, loss, train_prediction, learning_rate, merged], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            writer.add_summary(merged_output)
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.300922 learning rate: 10.000000
Minibatch perplexity: 27.14
v fcroqbl rbabhuq fminowif dck orohcmo jrh ehd xitgkdgfo eunettn tiomxmvcdt a ud
xcsvn rcxgtk pqub planviarsx atcntiznnqxexldlspjkteskh btq opoohxfjg laqjo h ktx
slntnas ducelrw o nsnqlyoxebrmc qnfdoexqe ef dxs eneyb kse prbhbxkibsb k ct injm
fxobzayf atsileawrbtpjko at rabyoewzipqtjz  t  wqhazetwhw ena l wd eha effhasv  
pl jpqcmrpnmsmlnnfuqrrtre iao nxdajjmohuynoe  ann cdkounpeoiiscclzpmehnb totrcll
Validation set perplexity: 20.34
Average loss at step 100: 2.585738 learning rate: 10.000000
Minibatch perplexity: 10.89
Validation set perplexity: 10.28
Average loss at step 200: 2.243923 learning rate: 10.000000
Minibatch perplexity: 8.56
Validation set perplexity: 8.58
Average loss at step 300: 2.112426 learning rate: 10.000000
Minibatch perplexity: 7.43
Validation set perplexity: 8.12
Average loss at step 400: 2.010247 learning rate: 10.000000
Minibatch perplexity: 7.73
Validation set per

# Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Answer 2
========

Haven't done anything yet but copy these code blocks (and concatenated a couple)

The plan:
1. Get it working using encodings for the single character case and compare to the previous
2. Modify to use bigrams

For #2 above, I could do the following options (assume alphabet input)
* input1=ab, targeted_output1=c  ; input2=bc, targeted_output2=d 
* input1=ab, targeted_output1=c  ; input2=cd, targeted_output2=e 
* input1=ab, targeted_output1=cd ; input2=cd, targeted_output2=ef


---

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('Data size %d' % len(text))

valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print('training data size: %s %s' % (train_size, train_text[:64]))
print('validation data size: %s %s' % (valid_size, valid_text[:64]))

vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))
print('vocabulary size: %s' % vocabulary_size)

Found and verified text8.zip


MemoryError: 

In [ ]:
batch_size=64
num_unrollings=10
embedding_size = 128  # size of the embeddings we will use for input


day_debug = False

# DAY
#
# This is important to understand.  Our NN needs a constant sized vector with each input.  We are
# providing that here.  As the video says, just as convolution lets us use the same weight parameters
# at different parts of the image, a recurrent neural net lets us use the same weights at different
# points in time (or rather, different points in the input sequence).
#
# The notion of "unrollings" is that a recurrent NN has it's output connected to it's input, but really
# the way to think about it is over time where the output of time t-1 is input to time t.  That way
# of looking at it is like "unrolling" the recurrent NN over time so it is understood more as a
# sequence of copies of the NN.  
# In this case, we are going to be feeding in sequences that are 10 long, so we will in effect
# create 10 LSTM cells (which are really just a NN) and hook the output of LSTM cell t with inputs
# from input_sub_t and also the output of LSTM cell t-1.

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
        if day_debug:
            print('BatchGenerator init: \n  textsize: %s\n  batch_size: %s\n  num_unrollings: %s' % 
                  (len(text), batch_size, num_unrollings))
            print("BatchGenerator init -- text starts with: %s" % text[:128])
            print("BatchGenerator init -- segment=%s" % segment)
            print("BatchGenerator init -- _cursor=%s" % self._cursor)
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data.
        It will be in the form of an embedding)"""
        # DAY
        # this is quite confusing, but batch ends up being  of dimension
        # (batch_size, vocabulary_size) -- so it is 1 char (one-hot encoded) from
        # every _cursor location (there are batch_size cursor locations.  
        # Since the cursor locations are not consecutive, these chars in batch
        # are not consecutive.  However, in the next step at next(), the 
        # char will be appended with the next consecutive char.
        # Uncomment the print line below to see this.
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        #print(batches2string([batch]))
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    # DAY
    # This mangles the real batch structure in the interest of readability, but
    # by doing so, makes your understanding of batches wrong.
    # See my 'honest_batches2string' below which gives you a better
    # understanding of the batch format.
    
    # batches has dimensions (num_unrollings, batch_size, vocabulary_size)
    s = [''] * batches[0].shape[0]  # batches[0].shape[0] will end up being same as batch_size
    for b in batches: # there will be num_unrollings of these...
        # each b (batch) is 64 by 27
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

def honest_batches2string(batches):
    import pprint
    output = []
    for b_index, b in enumerate(batches):  # there will be 'num_unrollings' of these
        output.append(list())
        for one_hot_index, one_hot in enumerate(b):  # there will be 'batch_size' of these
            output[b_index].append(characters([one_hot]))
    return pprint.pformat(output)
            
        

if day_debug:
    # DAY
    # Notice how this output is structured. Subsequent batches are related to each other in that
    # you can continue reading the text from batch1[0] to batch2[0]  (similarly from
    # batch1[20] to batch2[20])
    my_text = "abcdefghijklmnopqrstuvwxyz"
    my_batches = BatchGenerator(my_text.lower(), 4, 2)
    if False:
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
        print(my_batches.next())
    else:
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))
        print(honest_batches2string(my_batches.next()))

    my_text = "Four score and seven years ago our fathers brought forth on this continent, a new \
    nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. Now \
    we are engaged in a great civil war, testing whether that nation, or any nation so conceived and \
    so dedicated, can long endure. We are met on a great battle-field of that war. We have come to \
    dedicate a portion of that field, as a final resting place for those who here gave their lives that \
    that nation might live. It is altogether fitting and proper that we should do this.\
    But, in a larger sense, we can not dedicate -- we can not consecrate -- we can not hallow -- \
    this ground. The brave men, living and dead, who struggled here, have consecrated it, far above \
    our poor power to add or detract. The world will little note, nor long remember what we say here, \
    but it can never forget what they did here. It is for us the living, rather, to be dedicated here \
    to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for \
    us to be here dedicated to the great task remaining before us -- that from these honored dead we \
    take increased devotion to that cause for which they gave the last full measure of devotion -- that \
    we here highly resolve that these dead shall not have died in vain -- that this nation, under God, \
    shall have a new birth of freedom -- and that government of the people, by the people, for the people, \
    shall not perish from the earth."
    my_batches = BatchGenerator(my_text.lower(), 10, 5)
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    print(honest_batches2string(my_batches.next()))
    # Also, here is how a batch *really* looks (an batch_size array of one-hot encodings)
    temp = my_batches.next()
    print('Dimensions of the batch (num_unrollings, batch_size, vocab_size): (%d, %d, %d)' % 
          (len(temp), len(temp[0]), len(temp[0][0])))
    print('='*80)



train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print('WARNING: the following printed lines are not indicative of the structure that goes into the model')
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))


In [ ]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0  # prediction is in column format, so it must be indexed by [0]
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

_*DAY: Note that I have modified this to comment/understand*_

In [3]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    
    # My embedding here will simply be a 2D tensor -- the first
    # dimension will hold the one-hot encoding for a character (the index)
    # the second dimension will hold the embedding vector.
    vocabulary_embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

  
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1)) # [27, 64]
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))       # [64, 64]
    ib = tf.Variable(tf.zeros([1, num_nodes]))                                     # [1, 64]
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #notice the 'trainable=False'
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #notice the 'trainable=False'
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))  #DAY -  embedding_size?
    b = tf.Variable(tf.zeros([vocabulary_size]))  #DAY -  embedding_size?
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    # The LSTM
    #
    # In the code above, batch_size (bs) and num_nodes (nn) are both 64 (they don't have to be equal)
    # 27 is the vocabulary size.
    #
    # input_gate(bs, nn)   = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # forget_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # output_gate(bs, nn)  = sigmoid( input(bs, 27) * ix(27, nn) + output(bs, nn) * im(nn, nn) + ib(1, nn) )
    # update(bs, nn)       = tanh(    input(bs, 27) * cx(27, nn) + output(bs, nn) * cm(nn, nn) + cb(1, nn) )
    #
    # output(bs, nn) = output_gate(bs, nn) * tanh( state(bs, nn) )
    # state(bs, nn)  = forget_gate(bs, nn) * state(bs, nn)  +  input_gate(bs, nn) * update(bs, nn)

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            # These 11 elements of train_data will be pulled in from feed_dict
            # Note that usually we have seen feed_dict specified as {var_name: value}
            # but in this case, since these 11 array elements don't have a var_name, the 
            # feed_dict will use the tensorflow object as the key instead, i.e.
            # feed_dict={<tf.Tensor 'Placeholder_1:0' shape=(64, 27) dtype=float32>: value}
            # See below where the feed_dict is prepared before calling session.run.
            tf.placeholder(tf.float32, shape=[batch_size,embedding_size])) #this will be pulled in from feed_dict
    # train_data now has the shape (11, 64, 27), or (num_unrollings, batch_size, vocabulary_size)
    # and sequential text from the original text input is 'striped' across the first dimension (11)
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step. (Brilliant!)

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs: # since train_inputs is num_unrollings=10 long, this will create 10 LSTM cells
        #
        # the input i is not a single character or even a single one-hot -- it is 
        # 'batch_size' rows, each of which is a one-hot.  To get the character index from
        # the one-hot, use tf.argmax (since the only 1 in the one-hot will be the max)
        iembed = tf.nn.embedding_lookup(vocabulary_embeddings, tf.argmax(i, dimension=1))
        print('iembed = %s' % iembed)
        output, state = lstm_cell(iembed, output, state)
        outputs.append(output) # at each iteration of the lstm_cell, append the character it predicted to outputs.

    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    #   Note that all 10 unrollings are done before the optimizer comes in and looks at the
    #   output sequence of 10 chars vs. the label sequence of 10 chars and then calculates
    #   the gradients and adjusts the parameters.  Then in the next step another 10 characters
    #   will be predicted.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    # DAY: this clipping below is the hack to prevent exploding gradients 
    #(LSTM was the elegant solution to prevent vanishing gradient)
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    # (nothing here is triggered in the training)
    #     first, variables
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    #     reset zeros out saved_sample_output and saved_sample_state
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    #     Define one lstm_cell with no unrolling (will be used for sampling from the trained model)
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    #     Define the next prediction (but make sure dependencies are calculated first)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

NameError: name 'embedding_size' is not defined

Note that I've modified this with comments to understand

In [26]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next() #
        # batches is of dimension (num_unrollings, batch_size, vocabulary_size) (11, 64, 27)
        # where sequential text from the input is "striped" across the unrollings.  For 
        # example, if 'char1' stands for the first character in the original text (one-hot encoded), 
        # batches looks like this (assuming 'segment' is 15000):
        # [                                        # there are 'num_unrollings' rows
        #   [char1,  char15000,  char 30000, ...], # each row is 'batch_size'
        #   [char2,  char15001,  char 30001, ...],
        #   ...
        #   [char11, char15010,  char 30010, ...]
        # ]
        # when we call train_batches.next(), the next 'batches' will look like this:
        # [                                        # there are 'num_unrollings' rows
        #   [char11, char15010,  char 30010, ...], # each row is 'batch_size'
        #   [char12, char15011,  char 30011, ...],
        #   ...
        #   [char21, char15020,  char 30020, ...]
        # ]
        # it might look like a bug that the second 'batches' repeats char11, char1510, etc.
        # but it is not a bug.  in the first 'batches', char11 was included only as the label
        # needed for the 10th entry (char10). (LSTM takes char10 in as an input and expects char11
        # as the true label of the output).  So -- char11 was never put into the LSTM_cell in the
        # first 'batches' -- it is only used as a label, so it need to be included as the first
        # item in the second 'batches' so that in can now be an input into an LSTM cell.
        #
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
            # Normally we see feed_dict={var_name: value}, but here we don't have the var_names
            # for the training data batches in the graph definition (it is an array of tensors)
            # so instead, we use the tensorflow object itself (from the graph definition, in
            # train_data[i]) as the key in the feed_dict entries.
            
        _, l, predictions, lr = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())  # start with a random character
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.291867 learning rate: 10.000000
Minibatch perplexity: 26.89
k isfoa neneqwh oixuppe aiaedd pahovpu dlwpxv efpfaanmf hlos h ras etsc  icdmew 
ead pkil xemseie uzjgta qacenundwcajqivogo hpto tmrwcwali asqobzlzmixszzfk iytgf
tsfzshefiqxr hzisf  ox yiitohdpsigakfbgu kvsfs   itilvrlzesef  avsxw n mwsoqbsiz
ehz es  makdyva nari fal xdq  itimc f kckivwxef itveghylw fipirkfnpujsfhhrgez i 
a  rkmqe oaidah vocleoj tdrtij  olmuyihp pwtyyiqcizveh mpefm ocoqoxjxcwt  ektbez
Validation set perplexity: 20.11
Average loss at step 100: 2.588131 learning rate: 10.000000
Minibatch perplexity: 10.88
Validation set perplexity: 10.33
Average loss at step 200: 2.241012 learning rate: 10.000000
Minibatch perplexity: 8.44
Validation set perplexity: 8.58
Average loss at step 300: 2.105546 learning rate: 10.000000
Minibatch perplexity: 7.42
Validation set perplexity: 8.12
Average loss at step 400: 2.007310 learning rate: 10.000000
Minibatch perplexity: 7.50
Validation set per

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

---
Answer 3
========

---